In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 4

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def DRM(x):
    u_hat = model(x)
    uxx = torch.zeros(x.size()[0], dimension).cuda() 
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2    
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + pi**2*u_hat - f_temp)**2)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda()  
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda()  
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda() 
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda() 
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda() 
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda() 
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda() 
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda() 
    xd2[:, 3] = torch.ones(Nb)
    sa1 = (torch.sum((model(xa1)-u_ex(xa1))**2)/xa1.size()[0]).cuda() 
    sa2 = (torch.sum((model(xa2)-u_ex(xa2))**2)/xa1.size()[0]).cuda()  
    sb1 = (torch.sum((model(xb1)-u_ex(xb1))**2)/xa1.size()[0]).cuda()  
    sb2 = (torch.sum((model(xb2)-u_ex(xb2))**2)/xa1.size()[0]).cuda() 
    sc1 = (torch.sum((model(xc1)-u_ex(xc1))**2)/xa1.size()[0]).cuda()  
    sc2 = (torch.sum((model(xc2)-u_ex(xc2))**2)/xa1.size()[0]).cuda() 
    sd1 = (torch.sum((model(xd1)-u_ex(xd1))**2)/xa1.size()[0]).cuda()  
    sd2 = (torch.sum((model(xd2)-u_ex(xd2))**2)/xa1.size()[0]).cuda() 
    part_2 = sa1 + sa2 + sb1 + sb2 + sc1 + sc2 + sd1 + sd2
    
    lambda1 = 100.0
    return part_1 + lambda1 * part_2 / 8

In [10]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 10)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=4, out_features=10, bias=True)
  (linear1): Linear(in_features=10, out_features=10, bias=True)
  (linear2): Linear(in_features=10, out_features=10, bias=True)
  (linear3): Linear(in_features=10, out_features=10, bias=True)
  (linear4): Linear(in_features=10, out_features=10, bias=True)
  (linear5): Linear(in_features=10, out_features=10, bias=True)
  (linear6): Linear(in_features=10, out_features=10, bias=True)
  (linear_out): Linear(in_features=10, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [16]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_4D_Dirichlet_width10.npy", error_save)
np.save("DGM_relative_error_4D_Dirichlet_width10.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(2533.2115, device='cuda:0')
current relative error is:  tensor(1.8106, device='cuda:0')
current epoch is:  50
current loss is:  tensor(917.8340, device='cuda:0')
current relative error is:  tensor(0.9807, device='cuda:0')
current epoch is:  100
current loss is:  tensor(325.2602, device='cuda:0')
current relative error is:  tensor(0.5007, device='cuda:0')
current epoch is:  150
current loss is:  tensor(104.3178, device='cuda:0')
current relative error is:  tensor(0.2789, device='cuda:0')
current epoch is:  200
current loss is:  tensor(57.2987, device='cuda:0')
current relative error is:  tensor(0.2163, device='cuda:0')
current epoch is:  250
current loss is:  tensor(39.6543, device='cuda:0')
current relative error is:  tensor(0.1955, device='cuda:0')
current epoch is:  300
current loss is:  tensor(32.0050, device='cuda:0')
current relative error is:  tensor(0.1671, device='cuda:0')
current epoch is:  350
current loss is:  tensor(26.4757, dev

current epoch is:  3100
current loss is:  tensor(0.7247, device='cuda:0')
current relative error is:  tensor(0.0217, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(0.6998, device='cuda:0')
current relative error is:  tensor(0.0220, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(0.6779, device='cuda:0')
current relative error is:  tensor(0.0220, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(0.7048, device='cuda:0')
current relative error is:  tensor(0.0217, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(0.6738, device='cuda:0')
current relative error is:  tensor(0.0219, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(0.6391, device='cuda:0')
current relative error is:  tensor(0.0207, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(0.5943, device='cuda:0')
current relative error is:  tensor(0.0191, device='cuda:0')
current epoch is:  3450
current loss is:  tensor(0.5909, devic

current epoch is:  6200
current loss is:  tensor(0.1805, device='cuda:0')
current relative error is:  tensor(0.0124, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(0.1692, device='cuda:0')
current relative error is:  tensor(0.0105, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(0.1908, device='cuda:0')
current relative error is:  tensor(0.0132, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(0.1987, device='cuda:0')
current relative error is:  tensor(0.0133, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(0.2490, device='cuda:0')
current relative error is:  tensor(0.0171, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(0.1852, device='cuda:0')
current relative error is:  tensor(0.0128, device='cuda:0')
current epoch is:  6500
current loss is:  tensor(0.1512, device='cuda:0')
current relative error is:  tensor(0.0146, device='cuda:0')
current epoch is:  6550
current loss is:  tensor(0.1398, devic

current epoch is:  9300
current loss is:  tensor(0.1411, device='cuda:0')
current relative error is:  tensor(0.0140, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(0.0932, device='cuda:0')
current relative error is:  tensor(0.0102, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(0.0918, device='cuda:0')
current relative error is:  tensor(0.0090, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(0.1388, device='cuda:0')
current relative error is:  tensor(0.0153, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(0.0811, device='cuda:0')
current relative error is:  tensor(0.0080, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(0.0783, device='cuda:0')
current relative error is:  tensor(0.0082, device='cuda:0')
current epoch is:  9600
current loss is:  tensor(0.1452, device='cuda:0')
current relative error is:  tensor(0.0168, device='cuda:0')
current epoch is:  9650
current loss is:  tensor(0.0845, devic

current epoch is:  12350
current loss is:  tensor(0.0598, device='cuda:0')
current relative error is:  tensor(0.0074, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(0.0619, device='cuda:0')
current relative error is:  tensor(0.0081, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(0.1012, device='cuda:0')
current relative error is:  tensor(0.0137, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(0.1297, device='cuda:0')
current relative error is:  tensor(0.0139, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(0.0586, device='cuda:0')
current relative error is:  tensor(0.0069, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(0.0520, device='cuda:0')
current relative error is:  tensor(0.0068, device='cuda:0')
current epoch is:  12650
current loss is:  tensor(0.0641, device='cuda:0')
current relative error is:  tensor(0.0126, device='cuda:0')
current epoch is:  12700
current loss is:  tensor(0.053

current epoch is:  15400
current loss is:  tensor(0.0566, device='cuda:0')
current relative error is:  tensor(0.0093, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(0.0468, device='cuda:0')
current relative error is:  tensor(0.0072, device='cuda:0')
current epoch is:  15500
current loss is:  tensor(0.0881, device='cuda:0')
current relative error is:  tensor(0.0106, device='cuda:0')
current epoch is:  15550
current loss is:  tensor(0.0397, device='cuda:0')
current relative error is:  tensor(0.0060, device='cuda:0')
current epoch is:  15600
current loss is:  tensor(0.0557, device='cuda:0')
current relative error is:  tensor(0.0087, device='cuda:0')
current epoch is:  15650
current loss is:  tensor(0.0522, device='cuda:0')
current relative error is:  tensor(0.0076, device='cuda:0')
current epoch is:  15700
current loss is:  tensor(0.0455, device='cuda:0')
current relative error is:  tensor(0.0079, device='cuda:0')
current epoch is:  15750
current loss is:  tensor(0.053

current epoch is:  18450
current loss is:  tensor(0.0401, device='cuda:0')
current relative error is:  tensor(0.0053, device='cuda:0')
current epoch is:  18500
current loss is:  tensor(0.0517, device='cuda:0')
current relative error is:  tensor(0.0070, device='cuda:0')
current epoch is:  18550
current loss is:  tensor(0.0444, device='cuda:0')
current relative error is:  tensor(0.0087, device='cuda:0')
current epoch is:  18600
current loss is:  tensor(0.0488, device='cuda:0')
current relative error is:  tensor(0.0058, device='cuda:0')
current epoch is:  18650
current loss is:  tensor(0.0377, device='cuda:0')
current relative error is:  tensor(0.0069, device='cuda:0')
current epoch is:  18700
current loss is:  tensor(0.0341, device='cuda:0')
current relative error is:  tensor(0.0053, device='cuda:0')
current epoch is:  18750
current loss is:  tensor(0.0539, device='cuda:0')
current relative error is:  tensor(0.0069, device='cuda:0')
current epoch is:  18800
current loss is:  tensor(0.035